In [1]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas scikeras

INFO: pip is looking at multiple versions of scikeras to determine which version is compatible with other requirements. This could take a while.


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.optimizers import Adam ,RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
import logging


2024-06-21 23:48:17.303939: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 23:48:44.009384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 23:48:44.009449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 23:48:47.603068: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-21 23:48:54.175670: I tensorflow/core/platform/cpu_feature_guar

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

In [4]:
train_dir = 'images/training'
test_dir = 'images/testing'
val_dir = 'images/testing'
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


val_datagen = ImageDataGenerator(rescale=1.0/255.0)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2393 images belonging to 3 classes.
Found 597 images belonging to 3 classes.
Found 597 images belonging to 3 classes.


In [5]:
def create_baseline_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, l2_reg=0.001, activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    optimizer_instance = tf.keras.optimizers.get(optimizer)
    optimizer_instance.learning_rate = learning_rate

    model.compile(optimizer=optimizer_instance, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [7]:
baseline_model = create_baseline_model()

2024-06-21 23:50:09.331768: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 23:50:22.379060: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 23:50:22.379308: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [8]:
history_baseline = baseline_model.fit(train_generator, validation_data=test_generator, epochs=5, callbacks=[reduce_lr, early_stopping])

Epoch 1/5


2024-06-21 23:51:08.090834: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-06-21 23:51:39.565430: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f84795e7b30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-21 23:51:39.565468: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-06-21 23:51:39.773451: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1719013899.912181   10673 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


75/75 [==============================] - 329s 4s/step - loss: 3.1974 - accuracy: 0.6828 - val_loss: 3.3536 - val_accuracy: 0.5695 - lr: 0.0010
Epoch 2/5
16/75 [=====>........................] - ETA: 35s - loss: 2.8238 - accuracy: 0.7734

KeyboardInterrupt: 

In [12]:

from tensorflow.keras.optimizers.schedules import CosineDecay

batch_size = 32
epochs = 50
input_shape = (224, 224, 3)
num_classes = 3

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Simplified ResNet50 model
def create_simplified_resnet50_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    # Learning rate schedule
    initial_learning_rate = 0.001
    lr_schedule = CosineDecay(
        initial_learning_rate,
        decay_steps=1000,
        alpha=0.1
    )
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create and train the ResNet50 model
resnet50_model = create_simplified_resnet50_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

resnet50_history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[early_stopping]
)


Epoch 1/50
74/74 [==============================] - 64s 759ms/step - loss: 1.3611 - accuracy: 0.5044 - val_loss: 0.8034 - val_accuracy: 0.5122
Epoch 2/50
74/74 [==============================] - 54s 726ms/step - loss: 0.7839 - accuracy: 0.5049 - val_loss: 0.7498 - val_accuracy: 0.5191
Epoch 3/50
35/74 [=============>................] - ETA: 24s - loss: 0.7488 - accuracy: 0.5134

KeyboardInterrupt: 

In [13]:


# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Simplified EfficientNetB0 model
def create_simplified_efficientnet_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    # Learning rate schedule
    initial_learning_rate = 0.001
    lr_schedule = CosineDecay(
        initial_learning_rate,
        decay_steps=1000,
        alpha=0.1
    )
    
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Create and train the EfficientNetB0 model
efficientnet_model = create_simplified_efficientnet_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

efficientnet_history = efficientnet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[early_stopping]
)


16705208/16705208 [==============================] - 1s 0us/step
Epoch 1/50


2024-06-22 00:02:40.946393: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_3/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


74/74 [==============================] - 63s 769ms/step - loss: 1.7693 - accuracy: 0.4795 - val_loss: 1.0568 - val_accuracy: 0.5104
Epoch 2/50
74/74 [==============================] - 54s 732ms/step - loss: 0.9211 - accuracy: 0.5070 - val_loss: 0.8138 - val_accuracy: 0.5104
Epoch 3/50
74/74 [==============================] - 56s 751ms/step - loss: 0.7774 - accuracy: 0.5083 - val_loss: 0.7442 - val_accuracy: 0.5156
Epoch 4/50
74/74 [==============================] - 55s 745ms/step - loss: 0.7367 - accuracy: 0.4977 - val_loss: 0.7199 - val_accuracy: 0.5139
Epoch 5/50
 6/74 [=>............................] - ETA: 43s - loss: 0.7231 - accuracy: 0.5208

KeyboardInterrupt: 

In [ ]:

# Create the ResNet50 model
def create_resnet50_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    #Unfreeze the top layers of the base model
    for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers
        layer.trainable = True
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model
    opt = Adam(learning_rate=0.001)
    l2_reg=0.001
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    
    return model

model = create_resnet50_model()


'''
def create_resnet50_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, l2_reg=0.001, activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    optimizer_instance = tf.keras.optimizers.get(optimizer)
    optimizer_instance.learning_rate = learning_rate

    model.compile(optimizer=optimizer_instance, loss='categorical_crossentropy', metrics=['accuracy'])
    return model
'''

In [ ]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3,min_lr=1e-6)
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True, mode='min')

# Train the model
history_resnet50 = model.fit(
    train_generator,
    epochs=5,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint])
    

In [ ]:


# Create the EfficientNet model
def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Unfreeze the top layers of the base model
    for layer in base_model.layers[-30:]:  # Unfreeze the last 30 layers
        layer.trainable = True
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Compile the model with RMSprop optimizer
    opt = RMSprop(learning_rate=0.0001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

efficientnet_model = create_efficientnet_model()





In [ ]:
# Callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
checkpoint = ModelCheckpoint('best_efficientnet_model.h5', monitor='val_loss', save_best_only=True, mode='min', verbose=1)

# Train the model
history_efficientnet = efficientnet_model.fit(
    train_generator,
    epochs=5,  # Increase the number of epochs
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping, checkpoint]
)


In [14]:
def ensemble_predict(models, test_generator):
    X = np.concatenate([model.predict(test_generator) for model in models], axis=1)
    y = test_generator.classes
    
    meta_model = LogisticRegression(max_iter=1000)
    meta_model.fit(X, y)
    
    X_test = np.concatenate([model.predict(test_generator) for model in models], axis=1)
    y_pred = meta_model.predict(X_test)
    
    return y_pred

In [15]:
def get_predictions(model, data_generator):
    predictions = model.predict(data_generator)
    return (predictions > 0.5).astype(int)

In [17]:
baseline_preds = get_predictions(baseline_model, test_generator)
resnet50_preds = get_predictions(resnet50_model, test_generator)
efficientnet_preds = get_predictions(efficientnet_model, test_generator)

19/19 [==============================] - 9s 430ms/step


In [23]:

ensemble_preds = np.round((baseline_preds + resnet50_preds + efficientnet_preds) / 3)

In [25]:
import numpy as np
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback

class F1ScoreCallback(Callback):
    def __init__(self, val_data):
        self.validation_data = val_data

    def on_epoch_end(self, epoch, logs=None):
        val_data, val_labels = self.validation_data.next()
        val_predict = np.argmax(self.model.predict(val_data), axis=1)
        val_target = np.argmax(val_labels, axis=1)
        _val_f1 = f1_score(val_target, val_predict, average='macro')
        print(f' — val_f1: {_val_f1:.4f}')
        logs['val_f1'] = _val_f1

# Create and train the ResNet50 model
resnet50_model = create_simplified_resnet50_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
f1_score_callback = F1ScoreCallback(val_generator)

resnet50_history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[early_stopping, f1_score_callback]
)

# Create and train the EfficientNetB0 model
efficientnet_model = create_simplified_efficientnet_model()

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
f1_score_callback = F1ScoreCallback(val_generator)

efficientnet_history = efficientnet_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[early_stopping, f1_score_callback]
)

# After training, calculate F1 score on test data
test_generator = val_datagen.flow_from_directory(
    'path_to_test_data',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_data, test_labels = next(test_generator)
test_preds = resnet50_model.predict(test_data)
test_preds_labels = np.argmax(test_preds, axis=1)
test_labels = np.argmax(test_labels, axis=1)
f1 = f1_score(test_labels, test_preds_labels, average='macro')
print(f'ResNet50 Model F1 Score on Test Data: {f1}')

test_preds = efficientnet_model.predict(test_data)
test_preds_labels = np.argmax(test_preds, axis=1)
f1 = f1_score(test_labels, test_preds_labels, average='macro')
print(f'EfficientNetB0 Model F1 Score on Test Data: {f1}')


Epoch 1/50
1/1 [==============================] - 1s 1s/step- loss: 1.3668 - accuracy: 0.49
 — val_f1: 0.3043
74/74 [==============================] - 59s 762ms/step - loss: 1.3668 - accuracy: 0.4964 - val_loss: 0.8083 - val_accuracy: 0.4913 - val_f1: 0.3043
Epoch 2/50
38/74 [==============>...............] - ETA: 22s - loss: 0.8076 - accuracy: 0.5090

KeyboardInterrupt: 

In [24]:
from sklearn.metrics import f1_score
test_labels = test_generator.classes
f1 = f1_score(test_labels, ensemble_preds)
print(f'Ensemble Model F1 Score: {f1}')

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets

In [21]:
models = [baseline_model, resnet50_model, efficientnet_model]
ensemble_predictions = ensemble_predict(models, test_generator)

19/19 [==============================] - 7s 375ms/step


In [27]:
correct_labels = test_generator.classes
ensemble_accuracy = accuracy_score(correct_labels, ensemble_predictions)
print(f'Ensemble Accuracy: {ensemble_accuracy:.2f}')

Ensemble Accuracy: 0.54


In [8]:
input_shape = (224, 224, 3)
num_classes = 3
batch_size = 32
epochs = 10  # Adjust as necessary

In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf

def create_resnet50_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the layers of the pre-trained ResNet50 model
    for layer in base_model.layers:
        layer.trainable = False
    
    # Implementing learning rate scheduler
    def lr_scheduler(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)
    
    scheduler = LearningRateScheduler(lr_scheduler)
    
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model, scheduler


In [10]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow as tf

def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=3):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze the layers of the pre-trained EfficientNetB0 model
    for layer in base_model.layers:
        layer.trainable = False
    
    # Implementing learning rate scheduler
    def lr_scheduler(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)
    
    scheduler = LearningRateScheduler(lr_scheduler)
    
    model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model, scheduler


In [11]:
resnet50_model, resnet50_scheduler = create_resnet50_model(input_shape=input_shape, num_classes=num_classes)
efficientnet_model, efficientnet_scheduler = create_efficientnet_model(input_shape=input_shape, num_classes=num_classes)

2024-06-21 20:23:46.486407: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 20:24:02.730992: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 20:24:02.731281: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

94765736/94765736 [==============================] - 0s 0us/step


16705208/16705208 [==============================] - 0s 0us/step


In [13]:
resnet50_history = resnet50_model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // batch_size,
    callbacks=[resnet50_scheduler]
)


Epoch 1/10


2024-06-21 20:26:02.217403: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-06-21 20:26:39.291364: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f9768c00180 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-21 20:26:39.291420: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-06-21 20:26:39.490745: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1719001599.790054   16707 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - 372s 4s/step - loss: 0.8151 - accuracy: 0.4909 - val_loss: 0.6966 - val_accuracy: 0.5139 - lr: 0.0010
Epoch 2/10
74/74 [==============================] - 49s 659ms/step - loss: 0.7106 - accuracy: 0.5214 - val_loss: 0.6944 - val_accuracy: 0.4826 - lr: 0.0010
Epoch 3/10
74/74 [==============================] - 49s 657ms/step - loss: 0.7014 - accuracy: 0.4884 - val_loss: 0.7036 - val_accuracy: 0.5208 - lr: 0.0010
Epoch 4/10
74/74 [==============================] - 52s 699ms/step - loss: 0.6977 - accuracy: 0.5205 - val_loss: 0.6951 - val_accuracy: 0.4844 - lr: 0.0010
Epoch 5/10
74/74 [==============================] - 47s 629ms/step - loss: 0.6961 - accuracy: 0.5087 - val_loss: 0.6931 - val_accuracy: 0.5122 - lr: 0.0010
Epoch 6/10
14/74 [====>.........................] - ETA: 32s - loss: 0.6923 - accuracy: 0.5335

KeyboardInterrupt: 

In [ ]:

num_models = 3
for i in range(num_models):
    model = create_baseline_model(input_shape=input_shape, num_classes=num_classes)
    
    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=val_generator.n // batch_size,
        callbacks=[early_stopping]
    )
    
    # Save model weights
    model.save_weights(f'baseline_model_weights_{i}.h5')


Epoch 1/50


In [28]:
# Train multiple instances of the baseline model
num_models = 3
models = []
for i in range(num_models):
    model = create_baseline_model(input_shape=input_shape, num_classes=num_classes)
    # Load pre-trained weights if available, or train here
    model.load_weights(f'path_to_baseline_weights_{i}.h5')  # Ensure you have trained and saved these models
    models.append(model)

# Predict with all models
predictions = [model.predict(test_generator, steps=test_generator.n // batch_size, verbose=1) for model in models]

# Ensemble predictions (average the probabilities)
ensemble_preds = np.mean(predictions, axis=0)
ensemble_preds_labels = np.argmax(ensemble_preds, axis=1)

# Calculate accuracy
correct_labels = test_generator.classes
ensemble_accuracy = accuracy_score(correct_labels, ensemble_preds_labels)
print(f'Ensemble Accuracy: {ensemble_accuracy:.2f}')

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'path_to_baseline_weights_0.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)